In [6]:
import psycopg2
import csv

def export_csv():
    c = psycopg2.connect(database="mimic", user="mimic_demo", password="mimic_demo", host="172.16.34.1", port="5432")
    cursor = c.cursor()

    cursor.execute('SET search_path to mimiciii')

    cursor.execute("""
        SELECT n.row_id, n.chartdate, n.text, a.hospital_expire_flag, string_agg(DISTINCT d.icd9_code, '/') as icd9_codes
        FROM noteevents n
        INNER JOIN admissions a ON n.hadm_id = a.hadm_id
        LEFT JOIN diagnoses_icd d ON a.hadm_id = d.hadm_id
        WHERE n.category = 'Discharge summary'
        GROUP BY n.row_id, n.chartdate, n.text, a.hospital_expire_flag
        limit 10000
    """)

    with open('discharge_summary_notes_sample(3).csv', mode='w', newline='', encoding='utf-8') as file:
        w = csv.writer(file)
        w.writerow(['note_id', 'chartdate', 'note_text', 'hospital_expire_flag', 'icd9_codes'])
        for note_id, chartdate, note_text, hospital_expire_flag, icd9_codes in cursor:
            c_str = chartdate.strftime('%Y-%m-%dT%H:%M:%SZ')
            w.writerow([note_id, c_str, note_text, hospital_expire_flag, icd9_codes])

    cursor.close()
    c.close()

if __name__ == '__main__':
    export_csv()

In [3]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.
